Author: Ryan Timbrook (RTIMBROO)  
DATE: 12/3/2019 <br>
Topic: 

## 1. Objective:
-----------------------------------------------------------------------------------------------------

In [15]:
import pandas as pd
import numpy as np
import json
import os
from os import path
import fnmatch
import io
import re
import string
from datetime import date
from datetime import time
from datetime import datetime

In [2]:
# custome python packages
import rtimbroo_utils as br  

In [3]:
# set global properties
notebook_file_name = 'merge_datasets_to_master'
report_file_name = 'merge_datasets_to_master'
app_name = 'merge_datasets_to_master'
log_level = 10 # 10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL

# setup working directory structure
# set global properties
dataDir = './data'
logOutDir = './logs'
imageDir = './images'

# create base output directories if they don't exist
if not os.path.exists(logOutDir): os.mkdir(logOutDir)
if not os.path.exists(imageDir): os.mkdir(imageDir)
if not os.path.exists(dataDir): os.mkdir(dataDir)

In [4]:
# get a logger for troubleshooting / data exploration
logger = br.getFileLogger(logOutDir+'/',app_name,level=log_level)

In [5]:
coach = "coach_search_result_tweet_text_data.csv"
player = "player_search_result_tweet_text_data.csv"
team = "team_search_result_tweet_text_data.csv"

In [30]:
coach_df = pd.read_csv(f'{dataDir}/{coach}')
player_df =  pd.read_csv(f'{dataDir}/{player}')
team_df = pd.read_csv(f'{dataDir}/{team}')


coach_df = coach_df.sort_values(by=['id'], ascending=False)
player_df = player_df.sort_values(by=['id'], ascending=False)
team_df = team_df.sort_values(by=['id'], ascending=False)

In [31]:
logger.info(f'coach_df shape: {coach_df.shape} | player shape: {player_df.shape} | team shape: {team_df.shape}')
logger.info(f'coach_df unique ids: {len(np.unique(coach_df["id"]))} | player unique ids: {len(np.unique(player_df["id"]))} | team unique ids: {len(np.unique(team_df["id"]))}')

coach_df.head(5)

coach_df shape: (289, 11) | player shape: (24500, 11) | team shape: (24495, 11)
coach_df unique ids: 289 | player unique ids: 4882 | team unique ids: 4877


,id,created_at,date,time,user,text,favorite_count,year,month,day_of_month,day_of_week
17,1200324875115810816,Fri Nov 29 08:05:25 +0000 2019,2019-11-29,08:05:25,TimP103,Bill O'Brien is the most successful ex-Bill Be...,11.0,2019,11,29,4
18,1200086820928622592,Thu Nov 28 16:19:29 +0000 2019,2019-11-28,16:19:29,STERLING9798,"As always, Patriots stand in the way for Bill ...",361.0,2019,11,28,3
19,1200023580542676993,Thu Nov 28 12:08:11 +0000 2019,2019-11-28,12:08:11,monkey_viral,"As always, Patriots stand in the way for Bill ...",569.0,2019,11,28,3
20,1199968219286687745,Thu Nov 28 08:28:12 +0000 2019,2019-11-28,08:28:12,SVOFL,"As always, Patriots stand in the way for Bill ...",34.0,2019,11,28,3
21,1199961952556109824,Thu Nov 28 08:03:18 +0000 2019,2019-11-28,08:03:18,100UBF,"As always, Patriots stand in the way for Bill ...",7.0,2019,11,28,3


In [32]:
player_df.sort_values(by=['id'], ascending=False).head(5)

,id,created_at,date,time,user,text,favorite_count,year,month,day_of_month,day_of_week
24400,1200925257407827968,Sat Nov 30 23:51:08 +0000 2019,2019-11-30,23:51:08,JT_Evans97,You mean the same fan experts who could see La...,28503,2019,11,30,5
24000,1200925257407827968,Sat Nov 30 23:51:08 +0000 2019,2019-11-30,23:51:08,JT_Evans97,You mean the same fan experts who could see La...,28503,2019,11,30,5
24300,1200925257407827968,Sat Nov 30 23:51:08 +0000 2019,2019-11-30,23:51:08,JT_Evans97,You mean the same fan experts who could see La...,28503,2019,11,30,5
24200,1200925257407827968,Sat Nov 30 23:51:08 +0000 2019,2019-11-30,23:51:08,JT_Evans97,You mean the same fan experts who could see La...,28503,2019,11,30,5
24100,1200925257407827968,Sat Nov 30 23:51:08 +0000 2019,2019-11-30,23:51:08,JT_Evans97,You mean the same fan experts who could see La...,28503,2019,11,30,5


In [33]:
team_df.sort_values(by=['id'], ascending=False).head(5)

,id,created_at,date,time,user,text,favorite_count,year,month,day_of_month,day_of_week
1695,1200925546487504897,Sat Nov 30 23:52:17 +0000 2019,2019-11-30,23:52:17,miamidolphin12,RT SEVENTEEN!,1398,2019,11,30,5
1895,1200925546487504897,Sat Nov 30 23:52:17 +0000 2019,2019-11-30,23:52:17,miamidolphin12,RT SEVENTEEN!,1398,2019,11,30,5
1495,1200925546487504897,Sat Nov 30 23:52:17 +0000 2019,2019-11-30,23:52:17,miamidolphin12,RT SEVENTEEN!,1398,2019,11,30,5
1795,1200925546487504897,Sat Nov 30 23:52:17 +0000 2019,2019-11-30,23:52:17,miamidolphin12,RT SEVENTEEN!,1398,2019,11,30,5
1595,1200925546487504897,Sat Nov 30 23:52:17 +0000 2019,2019-11-30,23:52:17,miamidolphin12,RT SEVENTEEN!,1398,2019,11,30,5


In [41]:
player_df = player_df.drop_duplicates()
team_df = team_df.drop_duplicates()
coach_df = coach_df.drop_duplicates()

In [38]:
team_df.sort_values(by=['id'], ascending=False).head(5)

,id,created_at,date,time,user,text,favorite_count,year,month,day_of_month,day_of_week
1695,1200925546487504897,Sat Nov 30 23:52:17 +0000 2019,2019-11-30,23:52:17,miamidolphin12,RT SEVENTEEN!,1398,2019,11,30,5
1596,1200924401002303498,Sat Nov 30 23:47:43 +0000 2019,2019-11-30,23:47:43,DelindaTierney,"Since 2012, Tom Brady has won 7 straight games...",3148,2019,11,30,5
1897,1200922839135784960,Sat Nov 30 23:41:31 +0000 2019,2019-11-30,23:41:31,Apostleg180,NFL picks: Predictions for New England Patriot...,34,2019,11,30,5
1598,1200921166703157250,Sat Nov 30 23:34:52 +0000 2019,2019-11-30,23:34:52,BullenTiesha,Week 13 Power Rankings: 1) San Francisco 49er...,2574,2019,11,30,5
1499,1200920050686943233,Sat Nov 30 23:30:26 +0000 2019,2019-11-30,23:30:26,ishak548,WEEK 13 BETTING PREVIEWS Check out our bettin...,2567,2019,11,30,5


In [42]:
logger.info(f'coach_df shape: {coach_df.shape} | player shape: {player_df.shape} | team shape: {team_df.shape}')

coach_df shape: (289, 11) | player shape: (4933, 11) | team shape: (4926, 11)


In [43]:
coach = "coach_search_result_tweet_text_data_unique.csv"
player = "player_search_result_tweet_text_data_unique.csv"
team = "team_search_result_tweet_text_data_unique.csv"

#player_df.to_csv(f'{dataDir}/{player}')
#team_df.to_csv(f'{dataDir}/{team}')
#coach_df.to_csv(f'{dataDir}/{coach}')

In [46]:
nfl_type_coach = []
nfl_type_player = []
nfl_type_team = []

for i in range(0,coach_df.shape[0]):
    nfl_type_coach.append('coach')
logger.info(f'nfl_type_coach length: {len(nfl_type_coach)}')

for i in range(0,player_df.shape[0]):
    nfl_type_player.append('player')
logger.info(f'nfl_type_player length: {len(nfl_type_player)}')
    
for i in range(0,team_df.shape[0]):
    nfl_type_team.append('team')
logger.info(f'nfl_type_team lenght: {len(nfl_type_team)}')


nfl_type_coach length: 289
nfl_type_player length: 4933
nfl_type_team lenght: 4926


In [47]:
coach_df['nfl_type'] = nfl_type_coach
team_df['nfl_type'] = nfl_type_team
player_df['nfl_type'] = nfl_type_player

In [48]:
coach_df.head()

,id,created_at,date,time,user,text,favorite_count,year,month,day_of_month,day_of_week,nfl_type
17,1200324875115810816,Fri Nov 29 08:05:25 +0000 2019,2019-11-29,08:05:25,TimP103,Bill O'Brien is the most successful ex-Bill Be...,11.0,2019,11,29,4,coach
18,1200086820928622592,Thu Nov 28 16:19:29 +0000 2019,2019-11-28,16:19:29,STERLING9798,"As always, Patriots stand in the way for Bill ...",361.0,2019,11,28,3,coach
19,1200023580542676993,Thu Nov 28 12:08:11 +0000 2019,2019-11-28,12:08:11,monkey_viral,"As always, Patriots stand in the way for Bill ...",569.0,2019,11,28,3,coach
20,1199968219286687745,Thu Nov 28 08:28:12 +0000 2019,2019-11-28,08:28:12,SVOFL,"As always, Patriots stand in the way for Bill ...",34.0,2019,11,28,3,coach
21,1199961952556109824,Thu Nov 28 08:03:18 +0000 2019,2019-11-28,08:03:18,100UBF,"As always, Patriots stand in the way for Bill ...",7.0,2019,11,28,3,coach


In [49]:
search_date_ranges = [
    {
    1:('2019-09-01','2019-09-08'),      # week 1  --- dev_full_archive_url
    2:('2019-09-08','2019-09-15'),      # week 2  --- dev_full_archive_url
    3:('2019-09-15','2019-09-22'),      # week 3  --- dev_full_archive_url
    4:('2019-09-22','2019-09-29'),      # week 4  --- dev_full_archive_url
    5:('2019-09-29','2019-10-06'),      # week 5  --- dev_full_archive_url
    6:('2019-10-06','2019-10-13'),      # week 6  --- dev_full_archive_url
    7:('2019-10-13','2019-10-20'),      # week 7  --- dev_full_archive_url
    8:('2019-10-20','2019-10-27'),      # week 8  --- dev_full_archive_url
    9:('2019-10-27','2019-11-03'),      # week 9  --- dev_full_archive_url
    10:('2019-11-03','2019-11-10'),      # week 10 --- dev_30day_sandbox_url
    11:('2019-11-10','2019-11-17'),      # week 11 --- dev_30day_sandbox_url
    12:('2019-11-17','2019-11-24'),      # week 12 --- dev_30day_sandbox_url
    13:('2019-11-24','2019-12-01'),      # week 13 --- dev_30day_sandbox_url
    #14:('201912010000','201912080000'),     # week 14 
    #15:('201912080000','201912150000'),     # week 15
    #16:('201912150000','201912220000'),     # week 16
    #17:('201912220000','201912290000'),     # week 17
    }]

In [61]:
def match_weekly_schedule(df):
    nfl_schedule_wk = []
    
    for date in df['date']:
        logger.debug(date)
        df_date = datetime.strptime(date,'%Y-%m-%d')

        for w,dr in search_date_ranges[0].items():
            logger.debug(f'week: {w} | date range: {dr}')
            fromDate = datetime.strptime(dr[0],'%Y-%m-%d')
            toDate = datetime.strptime(dr[1],'%Y-%m-%d')
            if df_date >= fromDate and df_date < toDate:
                logger.debug(f'MATCH: --> {df_date} >= {fromDate} and {df_date} < {toDate}')
                nfl_schedule_wk.append(w)
                break
        #break

    print(nfl_schedule_wk)
    return nfl_schedule_wk

In [62]:
coach_df['nfl_schedule_wk'] = match_weekly_schedule(coach_df)
team_df['nfl_schedule_wk'] = match_weekly_schedule(team_df)
player_df['nfl_schedule_wk'] = match_weekly_schedule(player_df)

MATCH: --> 2019-11-29 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-29 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-28 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-28 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-28 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-28 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-28 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-28 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-28 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-28 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-28 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-28 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-27 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-27 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-27 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-27 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-27 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-27 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-27 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-27 00:00:00 < 2019-12-01 00:00:00


MATCH: --> 2019-09-30 00:00:00 >= 2019-09-29 00:00:00 and 2019-09-30 00:00:00 < 2019-10-06 00:00:00
MATCH: --> 2019-09-30 00:00:00 >= 2019-09-29 00:00:00 and 2019-09-30 00:00:00 < 2019-10-06 00:00:00
MATCH: --> 2019-09-30 00:00:00 >= 2019-09-29 00:00:00 and 2019-09-30 00:00:00 < 2019-10-06 00:00:00
MATCH: --> 2019-09-30 00:00:00 >= 2019-09-29 00:00:00 and 2019-09-30 00:00:00 < 2019-10-06 00:00:00
MATCH: --> 2019-09-18 00:00:00 >= 2019-09-15 00:00:00 and 2019-09-18 00:00:00 < 2019-09-22 00:00:00
MATCH: --> 2019-09-18 00:00:00 >= 2019-09-15 00:00:00 and 2019-09-18 00:00:00 < 2019-09-22 00:00:00
MATCH: --> 2019-09-16 00:00:00 >= 2019-09-15 00:00:00 and 2019-09-16 00:00:00 < 2019-09-22 00:00:00
MATCH: --> 2019-09-16 00:00:00 >= 2019-09-15 00:00:00 and 2019-09-16 00:00:00 < 2019-09-22 00:00:00
MATCH: --> 2019-09-12 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-12 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-12 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-12 00:00:00 < 2019-09-15 00:00:00


MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00
MATCH: --> 2019-11-30 00:00:00 >= 2019-11-24 00:00:00 and 2019-11-30 00:00:00 < 2019-12-01 00:00:00


MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00
MATCH: --> 2019-11-23 00:00:00 >= 2019-11-17 00:00:00 and 2019-11-23 00:00:00 < 2019-11-24 00:00:00


MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00
MATCH: --> 2019-11-09 00:00:00 >= 2019-11-03 00:00:00 and 2019-11-09 00:00:00 < 2019-11-10 00:00:00


...

MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00


MATCH: --> 2019-09-14 00:00:00 >= 2019-09-08 00:00:00 and 2019-09-14 00:00:00 < 2019-09-15 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00


MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00


MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00


MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
MATCH: --> 2019-09-07 00:00:00 >= 2019-09-01 00:00:00 and 2019-09-07 00:00:00 < 2019-09-08 00:00:00
[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,

In [63]:
coach_df.head()

,id,created_at,date,time,user,text,favorite_count,year,month,day_of_month,day_of_week,nfl_type,nfl_schedule_wk
17,1200324875115810816,Fri Nov 29 08:05:25 +0000 2019,2019-11-29,08:05:25,TimP103,Bill O'Brien is the most successful ex-Bill Be...,11.0,2019,11,29,4,coach,13
18,1200086820928622592,Thu Nov 28 16:19:29 +0000 2019,2019-11-28,16:19:29,STERLING9798,"As always, Patriots stand in the way for Bill ...",361.0,2019,11,28,3,coach,13
19,1200023580542676993,Thu Nov 28 12:08:11 +0000 2019,2019-11-28,12:08:11,monkey_viral,"As always, Patriots stand in the way for Bill ...",569.0,2019,11,28,3,coach,13
20,1199968219286687745,Thu Nov 28 08:28:12 +0000 2019,2019-11-28,08:28:12,SVOFL,"As always, Patriots stand in the way for Bill ...",34.0,2019,11,28,3,coach,13
21,1199961952556109824,Thu Nov 28 08:03:18 +0000 2019,2019-11-28,08:03:18,100UBF,"As always, Patriots stand in the way for Bill ...",7.0,2019,11,28,3,coach,13


In [65]:
coach = "coach_search_result_tweet_text_data_unique.csv"
player = "player_search_result_tweet_text_data_unique.csv"
team = "team_search_result_tweet_text_data_unique.csv"

player_df.to_csv(f'{dataDir}/{player}', index=False)
team_df.to_csv(f'{dataDir}/{team}', index=False)
coach_df.to_csv(f'{dataDir}/{coach}', index=False)

In [66]:
nfl_tweets_master_df = pd.DataFrame()

nfl_tweets_master_df = nfl_tweets_master_df.append(player_df, ignore_index=True)
nfl_tweets_master_df = nfl_tweets_master_df.append(team_df, ignore_index=True)
nfl_tweets_master_df = nfl_tweets_master_df.append(coach_df, ignore_index=True)

In [68]:
nfl_tweets_master_df.shape

(10148, 13)

In [69]:
nfl_tweets_master_df.to_csv(f'{dataDir}/nfl_tweets_master.csv', index=False)